In [41]:
import qiskit as qc
from qiskit import QuantumCircuit
import numpy as np

In [3]:
a = qc.QuantumCircuit(3,1)

a.h(0)
a.i(1)
a.cx(0,1)
a.i(2)


a.barrier()

a.y(0)
a.i(1)
a.h(2)

a.cx(1,2)
a.barrier()

a.i(0)
a.x(1)
a.i(2)

a.draw()

┌───┐      ░ ┌───┐      ░ ┌───┐
q_0: ┤ H ├──■───░─┤ Y ├──────░─┤ I ├
     ├───┤┌─┴─┐ ░ ├───┤      ░ ├───┤
q_1: ┤ I ├┤ X ├─░─┤ I ├──■───░─┤ X ├
     ├───┤└───┘ ░ ├───┤┌─┴─┐ ░ ├───┤
q_2: ┤ I ├──────░─┤ H ├┤ X ├─░─┤ I ├
     └───┘      ░ └───┘└───┘ ░ └───┘
c: 1/═══════════════════════════════

In [4]:
def parsing(QuantumCircuit):
    qasm = QuantumCircuit.qasm()
    lis = []
    lin = ""
    for s in qasm:
        if s == ";":
            lis.append(lin)
            lin=""
        elif s == "\n":
            pass
        else:
            lin += s
    
    lis.pop(0)
    lis.pop(0)
    #getting rid of the first two lines of the qasm code
    #could also make a while loop to pop() the lines that don't start with qreg
    
    if "qreg" in lis[0]:
        qreg=lis[0]
        numQubits=int(qreg[-2])
        lis.pop(0)
    else:
        print("There is an error in getting the number of qubits")
    
    if "creg" in lis[0]:
        lis.pop(0)
    
    easyGates = []
    numCycles = 1
    for i in range(len(lis)):
        line = lis[i]
        if "h" in line:
            easyGates.append("H")
        elif "x" in line and "c" not in line:
            easyGates.append("X")
        elif "y" in line:
            easyGates.append("Y")
        elif "z" in line:
            easyGates.append("Z")
        elif "t" in line:
            easyGates.append("T")
        elif "id" in line:
            easyGates.append("I")
        elif "barrier" in line:
            numCycles += 1
        elif "cx" in line:
            continue
        else:
            print(f"unexpected line or gate in qasm interpretation in line {i} ")
    
    controls = []
    targets = []
    
    for i in range(len(lis)):
        line = lis[i]
        if "cx" in line:
            for j in range(len(line)):
                if line[j] == "]":
                    a=line[j-1]
                    if j == 6:
                        controls.append(a)
                    elif j==11:
                        targets.append(a)
                    else:
                        print("something weird happening with cx")
    return numQubits, numCycles, easyGates, controls, targets

In [5]:
(numQubits, numCycles, easyGates, controls, targets)=parsing(a)

In [5]:
#can use structs to store info about circuits

In [7]:
def twirling_gates(numQubits, numCycles):
    twirling_set = ["X","Y","Z","I"]
    random_twirling = []
    for i in range(numQubits*(numCycles-1)):
        random_twirling.append(np.random.choice(twirling_set))
    for i in range(numQubits):
        random_twirling.append("I")
    return random_twirling

In [10]:
random_twirling = twirling_gates(numQubits, numCycles)
random_twirling

['I', 'I', 'Y', 'Z', 'I', 'Y', 'I', 'I', 'I']

In [11]:
def inverting_operators(random_twirling, targets, numQubits):
    target_indeces = []
    for i in range(len(targets)): #getting indeces of inverting_matrices that need to change
        target_indeces.append(str(int(targets[i])+ numQubits*i))
    
    inverting_gates = []
    for i in range(numQubits):
        inverting_gates.append("I")
    for i in range(len(random_twirling)-numQubits):
        if str(i) in target_indeces:
            #do multiplication formula:: Honestly, nevermind: just match with if state
            if random_twirling[i]=="X":
                inverting_gates.append("I")
            elif random_twirling[i]=="Y":
                inverting_gates.append("Z") #i think its a z rotation of maybe 180 degrees. Matrix is [i,0],[0,-i]
            elif random_twirling[i]=="Z":   #the fact that it's a z rotation makes sense w pen method.
                inverting_gates.append("Y") #not sure if correct. matrix is [0,-1],[1,0]
            elif random_twirling[i]=="I":
                inverting_gates.append("X")
            else:
                print("unrecognized gate before cx target qubit in prev. cycle")
        else:
            inverting_gates.append(random_twirling[i])
    return target_indeces, inverting_gates
        

In [12]:
(target_indeces, inverting_gates)=inverting_operators(random_twirling, targets, numQubits)
inverting_gates

['I', 'I', 'I', 'I', 'X', 'Y', 'Z', 'I', 'Z']

In [13]:
def gate2mat(gate):
    if gate == "X":
        gate_matrix = np.array([[0,1],
                                [1,0]])
    elif gate == "Y":
        gate_matrix = np.array([[0,-1j],
                                [1j,0]])
    elif gate == "Z":
        gate_matrix = np.array([[1,0],
                                [0,-1]])
    elif gate == "H":
        gate_matrix = 1/np.sqrt(2) *np.array([[1,1],
                                [1,-1]])
    elif gate == "I":
        gate_matrix = np.array([[1,0],
                                [0,1]])
    elif gate == "T":
        gate_matrix = np.array([[1,0],
                                [0,np.exp(1j*np.pi/4)]])
    else:
        print(f"matrix {gate} not recognized")
        gate_matrix = np.array([[0,0],[0,0]])
    return gate_matrix

In [14]:
gate2mat("H")

array([[ 0.70710678,  0.70710678],
       [ 0.70710678, -0.70710678]])

In [15]:
mat1=gate2mat("H")
mat2=gate2mat("X")

np.dot(mat1,mat2)

array([[ 0.70710678,  0.70710678],
       [-0.70710678,  0.70710678]])

In [16]:
def reduction(easyGates, random_twirling, inverting_gates, numQubits, numCycles):
    compiled_gates = np.zeros((numQubits*numCycles,2,2),dtype = 'complex_')
    
    for i in range(numQubits*numCycles):
        mat1=gate2mat(inverting_gates[i])
        mat2=gate2mat(easyGates[i])
        mat3=gate2mat(random_twirling[i])
        a=np.dot(mat2,mat1)
        compiled_gates[i]=np.dot(mat3,a)
    
    #want to return an array, matrix of 2x2x(however 
    #many easy gates we have in the circuit= numQubits*numCycles)
    return compiled_gates

In [17]:
(compiled_gates)=reduction(easyGates, random_twirling, inverting_gates, numQubits, numCycles)
compiled_gates

array([[[ 0.70710678+0.j,  0.70710678+0.j],
        [ 0.70710678+0.j, -0.70710678+0.j]],

       [[ 1.        +0.j,  0.        +0.j],
        [ 0.        +0.j,  1.        +0.j]],

       [[ 0.        +0.j,  0.        -1.j],
        [ 0.        +1.j,  0.        +0.j]],

       [[ 0.        +0.j,  0.        -1.j],
        [ 0.        -1.j, -0.        +0.j]],

       [[ 0.        +0.j,  1.        +0.j],
        [ 1.        +0.j,  0.        +0.j]],

       [[-0.70710678+0.j, -0.70710678+0.j],
        [-0.70710678+0.j,  0.70710678+0.j]],

       [[ 1.        +0.j,  0.        +0.j],
        [ 0.        +0.j, -1.        +0.j]],

       [[ 0.        +0.j,  1.        +0.j],
        [ 1.        +0.j,  0.        +0.j]],

       [[ 1.        +0.j,  0.        +0.j],
        [ 0.        +0.j, -1.        +0.j]]])

In [28]:
#turn matrix into gate: will need to know how to generalize the forms of the
# possible matrices
def mat2gate(a): #given a 2x2 np.array()
    gate=[]
    gateList = ["X", "Y", "Z", "H", "I", "T", "H","H","H","H","H","H","H","X","X","X"]
    
    X = np.array([[0,1],[1,0]])
    Y = np.array([[0,-1j],[1j,0]])
    Z = np.array([[1,0],[0,-1]])
    H = 1/np.sqrt(2) *np.array([[1,1],[1,-1]])
    I = np.array([[1,0],[0,1]])
    T = np.array([[1,0],[0,np.exp(1j*np.pi/4)]])
    
    #extra gates I've been adding after running code:
    H2 = 1/np.sqrt(2)*np.array([[-1,-1],[-1,1]])
    H3 = 1/np.sqrt(2)*np.array([[-1j,-1j],[-1j,1j]])
    H4 = 1/np.sqrt(2) *np.array([[1j,1j],[1j,-1j]])
    H5 = 1/np.sqrt(2)*np.array([[1,-1],[1,1]])
    H6 = 1/np.sqrt(2) *np.array([[-1,1],[-1,-1]])
    H7 = 1/np.sqrt(2)*np.array([[1j,-1j],[1j,1j]])
    H8 = 1/np.sqrt(2) *np.array([[-1j,1j],[-1j,-1j]])
    
    X2 = np.array([[0,-1j],[-1j,0]])
    X3 = np.array([[0,-1],[-1,0]])
    X4 = np.array([[0,1j],[1j,0]])
    
    matList = [X, Y, Z, H, I, T, H2,H3,H4,H5,H6,H7,H8,X2,X3,X4] #only works for these specific definitions
    for i,item in enumerate(matList):
        if (a == item).all():
            gate = gateList[i]
    if gate == []:
        print(f"the following gate needs to be defined:{a}")
    return gate

In [26]:
def instructions(compiled_gates):
    randEZgates = []
    for i in range(len(compiled_gates)):
        randEZgates.append(mat2gate(compiled_gates[i]))
        
    return randEZgates

In [29]:
randEZgates = instructions(compiled_gates)
print(randEZgates)
print(random_twirling)
print(inverting_gates)
a.draw()

['H', 'I', 'Y', 'X', 'X', 'H', 'Z', 'X', 'Z']
['I', 'I', 'Y', 'Z', 'I', 'Y', 'I', 'I', 'I']
['I', 'I', 'I', 'I', 'X', 'Y', 'Z', 'I', 'Z']


┌───┐      ░ ┌───┐      ░ ┌───┐
q_0: ┤ H ├──■───░─┤ Y ├──────░─┤ I ├
     ├───┤┌─┴─┐ ░ ├───┤      ░ ├───┤
q_1: ┤ I ├┤ X ├─░─┤ I ├──■───░─┤ X ├
     ├───┤└───┘ ░ ├───┤┌─┴─┐ ░ ├───┤
q_2: ┤ I ├──────░─┤ H ├┤ X ├─░─┤ I ├
     └───┘      ░ └───┘└───┘ ░ └───┘
c: 1/═══════════════════════════════

In [66]:
def final_product(randEZgates, controls, targets, numQubits):
    qc=QuantumCircuit(numQubits)
    for j in range(len(targets)):
        for i in range(numQubits):
            var = randEZgates[i+j*numQubits]
            if var == "X":
                qc.x(i)
            elif var == "Y":
                qc.y(i)
            elif var == "Z":
                qc.z(i)
            elif var == "H":
                qc.h(i)
            elif var == "I":
                qc.id(i)
            elif var == "T":
                qc.t(i)
            else:
                print(f"randEXgates has unexpected gate from mat2gate function at index {i}")
            #all the other gates we could possibly get, including paramaterized gates somehow
        qc.cx(int(controls[j]),int(targets[j]))
        qc.barrier()
    
    for i in range(len(targets)*numQubits,len(randEZgates)):
        var = randEZgates[i]
        if var == "X":
            qc.x(i-len(targets)*numQubits)
        elif var == "Y":
            qc.y(i-len(targets)*numQubits)
        elif var == "Z":
            qc.z(i-len(targets)*numQubits)
        elif var == "H":
            qc.h(i-len(targets)*numQubits)
        elif var == "I":
            qc.id(i-len(targets)*numQubits)
        elif var == "T":
            qc.t(i-len(targets)*numQubits)
        else:
            print(f"randEXgates has unexpected gate from mat2gate function at index {i-len(targets)*numQubits-1}")
    #one final loop for last round of randEZgates (not attached to hard gate cycle)
    return qc

In [67]:
randEZgates

['H', 'I', 'Y', 'X', 'X', 'H', 'Z', 'X', 'Z']

In [68]:
qc = final_product(randEZgates, controls, targets, numQubits)
qc.draw()

┌───┐      ░ ┌───┐      ░ ┌───┐
q_0: ┤ H ├──■───░─┤ X ├──────░─┤ Z ├
     ├───┤┌─┴─┐ ░ ├───┤      ░ ├───┤
q_1: ┤ I ├┤ X ├─░─┤ X ├──■───░─┤ X ├
     ├───┤└───┘ ░ ├───┤┌─┴─┐ ░ ├───┤
q_2: ┤ Y ├──────░─┤ H ├┤ X ├─░─┤ Z ├
     └───┘      ░ └───┘└───┘ ░ └───┘

In [34]:
hey = qc.QuantumCircuit(2)
hey.t(1)
hey.draw()

q_0: ─────
     ┌───┐
q_1: ┤ T ├
     └───┘

In [24]:
mat1=gate2mat("Y")
mat2=gate2mat("H")
earlier = np.dot(mat2,mat1)
mat3=gate2mat("Z")
earlier
np.dot(mat3,earlier) #-> this gate corresponds to an H gate, simply

array([[0.+0.70710678j, 0.-0.70710678j],
       [0.+0.70710678j, 0.+0.70710678j]])

In [ ]:
#from second random twirling run:
# we also need the Z*Y (corresponds to X), and Y*H*Y (corresponds to H but vector rotated about itself 180 degrees, 
# but that would require more gates// wrong, that would just be a Y90, this is simply an H)

In [159]:
#using the newly made easy gates (from the randomized compiling) and the original 
# hard gates, return a quantum circuit that has the same # of cycles and gates as 
# the bare circuit (and also include barriers)